# Blackmatter Ransomware

Sample: `22d7d67c3af10b1a37f277ebabe2d1eb4fd25afbd6437d4377400e148bcc08d6`

References:
- [Malware Bazaar Sample](https://bazaar.abuse.ch/sample/22d7d67c3af10b1a37f277ebabe2d1eb4fd25afbd6437d4377400e148bcc08d6/)
- [ChuongDong Blog](https://chuongdong.com/reverse%20engineering/2021/09/05/BlackMatterRansomware/)
- [CARBON SPIDER Attribution](https://www.youtube.com/watch?v=PAG3M7mWT2c&t=8192s)

## Helper Functions

In [4]:
def unhex(hex_string):
    import binascii
    if type(hex_string) == str:
        return binascii.unhexlify(hex_string.encode('utf-8'))
    else:
        return binascii.unhexlify(hex_string)

def tohex(data):
    import binascii
    if type(data) == str:
        return binascii.hexlify(data.encode('utf-8'))
    else:
        return binascii.hexlify(data)

## API Hashing

In [6]:
def ror(value, count=1, base=8):
    value = (value >> count | value << (base - count)) & (2**base - 1)
    return value


In [9]:
# Example hashing ntdll.dll

out = 0

for i in 'ntdll.dll\x00':
    out = (ord(i) + ror(out, count=13, base=32)) & 0xffffffff

print(hex(out))

0x411677b7


## Config Decryption

### APLib 

Credit: [Sandor Nemes (snemes)](https://github.com/snemes/aplib/blob/master/aplib.py)

In [35]:
import struct
from binascii import crc32
from io import BytesIO

__all__ = ['APLib', 'decompress']
__version__ = '0.6'
__author__ = 'Sandor Nemes'


class APLib(object):

    __slots__ = 'source', 'destination', 'tag', 'bitcount', 'strict'

    def __init__(self, source, strict=True):
        self.source = BytesIO(source)
        self.destination = bytearray()
        self.tag = 0
        self.bitcount = 0
        self.strict = bool(strict)

    def getbit(self):
        # check if tag is empty
        self.bitcount -= 1
        if self.bitcount < 0:
            # load next tag
            self.tag = ord(self.source.read(1))
            self.bitcount = 7

        # shift bit out of tag
        bit = self.tag >> 7 & 1
        self.tag <<= 1

        return bit

    def getgamma(self):
        result = 1

        # input gamma2-encoded bits
        while True:
            result = (result << 1) + self.getbit()
            if not self.getbit():
                break

        return result

    def depack(self):
        r0 = -1
        lwm = 0
        done = False

        try:

            # first byte verbatim
            self.destination += self.source.read(1)

            # main decompression loop
            while not done:
                if self.getbit():
                    if self.getbit():
                        if self.getbit():
                            offs = 0
                            for _ in range(4):
                                offs = (offs << 1) + self.getbit()

                            if offs:
                                self.destination.append(self.destination[-offs])
                            else:
                                self.destination.append(0)

                            lwm = 0
                        else:
                            offs = ord(self.source.read(1))
                            length = 2 + (offs & 1)
                            offs >>= 1

                            if offs:
                                for _ in range(length):
                                    self.destination.append(self.destination[-offs])
                            else:
                                done = True

                            r0 = offs
                            lwm = 1
                    else:
                        offs = self.getgamma()

                        if lwm == 0 and offs == 2:
                            offs = r0
                            length = self.getgamma()

                            for _ in range(length):
                                self.destination.append(self.destination[-offs])
                        else:
                            if lwm == 0:
                                offs -= 3
                            else:
                                offs -= 2

                            offs <<= 8
                            offs += ord(self.source.read(1))
                            length = self.getgamma()

                            if offs >= 32000:
                                length += 1
                            if offs >= 1280:
                                length += 1
                            if offs < 128:
                                length += 2

                            for _ in range(length):
                                self.destination.append(self.destination[-offs])

                            r0 = offs

                        lwm = 1
                else:
                    self.destination += self.source.read(1)
                    lwm = 0

        except (TypeError, IndexError):
            if self.strict:
                raise RuntimeError('aPLib decompression error')

        return bytes(self.destination)

    def pack(self):
        raise NotImplementedError


def aplib_decompress(data, strict=False):
    packed_size = None
    packed_crc = None
    orig_size = None
    orig_crc = None
    if data.startswith(b'AP32') and len(data) >= 24:
        # data has an aPLib header
        header_size, packed_size, packed_crc, orig_size, orig_crc = struct.unpack_from('=IIIII', data, 4)
        data = data[header_size : header_size + packed_size]
    if strict:
        if packed_size is not None and packed_size != len(data):
            raise RuntimeError('Packed data size is incorrect')
        if packed_crc is not None and packed_crc != crc32(data):
            raise RuntimeError('Packed data checksum is incorrect')
    result = APLib(data, strict=strict).depack()
    if strict:
        if orig_size is not None and orig_size != len(result):
            raise RuntimeError('Unpacked data size is incorrect')
        if orig_crc is not None and orig_crc != crc32(result):
            raise RuntimeError('Unpacked data checksum is incorrect')
    return result

### Extract Config

The Blackmatter config is stored in the PE resource section `.rsrc`. 

The first `DWORD` of the resource is the seed for an lcg that closely matches the zipcrypto lcg with constant `0x8088405`.

The second `DWORD` is the size of the encrypted config and is followed by the encrypted config data.

In [48]:
import struct
import pefile

RANSOMWARE_FILE = r'/tmp/blackmatter.bin'
data = open(RANSOMWARE_FILE, 'rb').read()
pe = pefile.PE(data = data)

# Get resource data
r_data = None
for s in pe.sections:
    if b'rsrc' in s.Name:
        r_data = s.get_data()
        
# Parse data from resource
seed = struct.unpack('<I',r_data[:4])[0]
data_size = struct.unpack('<I',r_data[4:8])[0]
enc_data = r_data[8:]

print("Seed: %s" % hex(seed))
print("Size: %d" % data_size)


Seed: 0xffcaa1ea
Size: 3487


In [93]:

def decrypt(enc_data, data_size, seed):
    fixed = seed
    decrypted = bytearray()
    for i in range(data_size-1):
        if i & 3 == 0:
            next_value = struct.unpack('<I',enc_data[i:i+4])[0]
            seed = (0x8088405 * seed + 1) & 0xffffffff
            rnd = ((seed * fixed) >> 32) & 0xffffffff
            dw = next_value ^ rnd
        decrypted.append((dw >> ((i & 3) * 8)) & 0xff)
    return decrypted

ap_data = decrypt(enc_data, data_size, seed)
ptxt_data = aplib_decompress(ap_data)
ptxt_data

b'\x87\x19\xa80\xf4\xba\x94\x94\x92\x91X+fT\xf9l\x96\xd9\xa0\xf4A\x9fR\xf3g\xcf.\x19\xb9\xc9Z\x9bp\x91\xcb\xef\xaf\xbeZ\xe3\x9d\xae(X\x94Y\n\x8d\xb8\xb7d\xe5r\xfa\xb5#FF\xf8e\x9a\xda/\xbd\x8c7\xbf\xdd\xd6\x07\x97\xa5\xad\x9d\xad-\xed7\x96\x9d\x17\x9e\xa4\xadL\x19\x80\xd0\xe7\x0b\x05bA\xd3%\xe1\x8b\xeb\\\xc4\x92_\xa5j\xbf\x81\x0f\x91ny2\xd0\x16\xa8n:\xd9wI\xe7_\x901\x11K\x06\x0bVQ$x\xc0\x8d\xad\xa2\xaf\x19\xe4\x98\x08\xfb\xda[\x0b\xa6\xf30\xb0\x9c\xd4{O\xb9!Ox6\xaaF\xad\x00\x01\x01\x01\x01\x01\x01\x01$\x00\x00\x00\xa1\x00\x00\x00\xe2\x00\x00\x00\x00\x00\x00\x00\xf3\x01\x00\x00\xc8\x04\x00\x009\x05\x00\x002\x06\x00\x00\x1f\x07\x00\x00ro4BrnX5Zms1fmgmp9Hypi0hCgPduMrclWUIq05OADb1eHAmezreXJI46rfXbELjszc67ztiIrrUJUtMlON1LsA7puHNgfKMOAvLUpTmZlNYac7GNXnwBwAAAAB=\x00UqLSghWqzIY3WZfbVqvI/NH3zsibCQc59aY6wgDsa4SWrgzwNariy+RXqoUAAAAA\x00k8UWrwAbmN9xl+JkwBxI3YAbWNsAHijNQBgQycAYkOlAGKDHgBywywAdgOfAHIDdgBww48AcGN3AGHjVQBnI2cAYwOHxlLpKMZbiSgAZEOMAHMDxABuA04CrnsnAG2jnABtgy9u3a0rAGyDdwByI4cAYmNUAHQjTQBpw

In [72]:
import base64
ptr = 0
rsa_data = ptxt_data[ptr:128]
ptr += 128
affiliate_id_data = ptxt_data[ptr:ptr+32]
ptr+= 32
config_flags = ptxt_data[ptr:ptr+22]
ptr+= 8
config_values_offset = struct.unpack('<I',ptxt_data[ptr:ptr+4])[0]
config_values_buffer = ptxt_data[ptr+config_values_offset:]
config_values = []
for c in config_values_buffer.split(b'\x00'):
    config_values.append(base64.b64decode(c))

In [97]:
def is_ascii(s):
    return all(c < 128 for c in s)

print("RSA: %r\n" % rsa_data)
print("Affiliate ID: %r\n" % affiliate_id_data)
print("Flags: %s\n" % tohex(config_flags))
for c in config_values:
    if not is_ascii(c):
        c = new_data =  decrypt(c,len(c), seed)
    print("%s\n" % b' | '.join([s.replace(b'\x00',b'') for s in c.split(b'\x00\x00')]))
    

RSA: b'\x87\x19\xa80\xf4\xba\x94\x94\x92\x91X+fT\xf9l\x96\xd9\xa0\xf4A\x9fR\xf3g\xcf.\x19\xb9\xc9Z\x9bp\x91\xcb\xef\xaf\xbeZ\xe3\x9d\xae(X\x94Y\n\x8d\xb8\xb7d\xe5r\xfa\xb5#FF\xf8e\x9a\xda/\xbd\x8c7\xbf\xdd\xd6\x07\x97\xa5\xad\x9d\xad-\xed7\x96\x9d\x17\x9e\xa4\xadL\x19\x80\xd0\xe7\x0b\x05bA\xd3%\xe1\x8b\xeb\\\xc4\x92_\xa5j\xbf\x81\x0f\x91ny2\xd0\x16\xa8n:\xd9wI\xe7_\x901\x11K\x06\x0bV'

Affiliate ID: b'Q$x\xc0\x8d\xad\xa2\xaf\x19\xe4\x98\x08\xfb\xda[\x0b\xa6\xf30\xb0\x9c\xd4{O\xb9!Ox6\xaaF\xad'

Flags: b'000101010101010124000000a1000000e20000000000'

b'\xc4\xe2\x95wo\xed9>\x10\xdb\x16\xd9.\xa5\x01\xcc\xaeP\xc4t\xdc\xb0\xbc\xf5\xe2\x860\xde\x9e\x1b\x81Q\x91\x0e\xc9\x7f\xe4Y\xe6\x8aX*~\x0e\x84W\xb0\xed\x1aA\xe9f\x10\x0fq\xbd\x1eg\x02z\xa7\xac[\xa9z\x9el\x1b\xe1\x1f\xb8\xb1,\xbepPX:\xcc\xb1L\xc0\xc0\x13\x943\x99\x05\x97\x14C'

b'8\xceF[\x0f\xbe\x93\xd3\x12\xfc\xe9$\xdf\xdf;\x96R\x86\xbf\x9a\x01q\x10\x82E\x02\xb7\xd0\xb9\xea\xe3\xf2\xd8\xb5\xa9\xaa\xc9\xda\x1d.E><S;\xf2'

b"\xf9\xa9\x82v\x1